In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data_set=pd.read_csv('iris_dataset.csv')

In [3]:
train=data_set.iloc[:,:-1]

In [4]:
data_set

,Unnamed: 0,0,1,2,3,Type
0,0,5.1,3.5,1.4,0.2,0
1,1,4.9,3.0,1.4,0.2,0
2,2,4.7,3.2,1.3,0.2,0
3,3,4.6,3.1,1.5,0.2,0
4,4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...,...
145,145,6.7,3.0,5.2,2.3,2
146,146,6.3,2.5,5.0,1.9,2
147,147,6.5,3.0,5.2,2.0,2
148,148,6.2,3.4,5.4,2.3,2


Number of null in dataset

In [5]:
data_set.isnull().sum()

Unnamed: 0    0
0             2
1             1
2             5
3             2
Type          0
dtype: int64

filling the null values

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
train=imputer.fit_transform(train)

In [8]:
train=pd.DataFrame(train)

In [9]:
train.isnull().sum()

0    0
1    0
2    0
3    0
4    0
dtype: int64

convert categorical into numercal

In [10]:
from sklearn.preprocessing import OrdinalEncoder

In [11]:
ordinal_encoder = OrdinalEncoder()
train=ordinal_encoder.fit_transform(data_set)

In [12]:
data_set

,Unnamed: 0,0,1,2,3,Type
0,0,5.1,3.5,1.4,0.2,0
1,1,4.9,3.0,1.4,0.2,0
2,2,4.7,3.2,1.3,0.2,0
3,3,4.6,3.1,1.5,0.2,0
4,4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...,...
145,145,6.7,3.0,5.2,2.3,2
146,146,6.3,2.5,5.0,1.9,2
147,147,6.5,3.0,5.2,2.0,2
148,148,6.2,3.4,5.4,2.3,2


1. LabelEncoder:

In [13]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels = ['cat', 'dog', 'bird']

encoded_labels = label_encoder.fit_transform(labels)

print("Original Labels:", labels)
print("Encoded Labels:", encoded_labels)


Original Labels: ['cat', 'dog', 'bird']
Encoded Labels: [1 2 0]


2. OrdinalEncoder:

In [14]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
data = [['low', 10], ['medium', 20], ['high', 30]]

encoded_data = ordinal_encoder.fit_transform(data)

print("Original Data:")
for row in data:
    print(row)

print("\nEncoded Data:")
for row in encoded_data:
    print(row)


Original Data:
['low', 10]
['medium', 20]
['high', 30]

Encoded Data:
[1. 0.]
[2. 1.]
[0. 2.]


3. OneHotEncoder:

In [15]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()
data = [['red', 1], ['green', 2], ['blue', 3]]

encoded_data = onehot_encoder.fit_transform(data).toarray()

print("Original Data:")
for row in data:
    print(row)

print("\nEncoded Data:")
for row in encoded_data:
    print(row)


Original Data:
['red', 1]
['green', 2]
['blue', 3]

Encoded Data:
[0. 0. 1. 1. 0. 0.]
[0. 1. 0. 0. 1. 0.]
[1. 0. 0. 0. 0. 1.]


# convert into dummy variables (0 and 1)
  |   0     |    1     |     2   |  #column names
  |------------------------------
  |   0     |    0     |     1   |  #row 1
  |   1     |    0     |     0   |  #row 2
  |   0     |    1     |     0   |  #row 3

In [16]:
#Catgorical data  
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_x= LabelEncoder()
x[:, 3]= labelencoder_x.fit_transform(x[:,3])
onehotencoder= OneHotEncoder(categorical_features= [3])
x= onehotencoder.fit_transform(x).toarray()

NameError: name 'x' is not defined

# Backward elimination

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.metrics import accuracy_score, confusion_matrix
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Function to calculate VIF
def calculate_vif(data):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = data.columns
    vif_data["VIF"] = [variance_inflation_factor(data.values, i) for i in range(data.shape[1])]
    return vif_data

# Load a different dataset to avoid issues with perfect separation
iris = datasets.load_iris()
X = iris.data
y = (iris.target == 0).astype(int)  # Binary classification for setosa

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Add a constant term to the predictors matrix
X_train_scaled = sm.add_constant(X_train_scaled)

# Step 1: Select Significance Level (SL)
SL = 0.05

# Initial check for multicollinearity using VIF
initial_vif_data = calculate_vif(pd.DataFrame(X_train_scaled[:, 1:], columns=iris.feature_names))
print("Initial VIF:")
print(initial_vif_data)

# Step 2: Fit the Complete Model
try:
    model = sm.Logit(y_train, X_train_scaled).fit()
except Exception as e:
    print("Exception:", e)

# Step 3-5: Backward Elimination Loop
selected_features = list(range(X_train_scaled.shape[1]))
while np.max(model.pvalues) > SL:
    max_p_value_index = np.argmax(model.pvalues[1:]) + 1  # Exclude the constant term
    selected_features.remove(max_p_value_index)  # Remove the predictor with the highest p-value
    X_train_scaled = sm.add_constant(X_train_scaled[:, selected_features])  # Rebuild the matrix
    try:
        model = sm.Logit(y_train, X_train_scaled).fit()  # Rebuild the model
    except Exception as e:
        print("Exception:", e)
        break

# Final Model Summary
print(model.summary())

# Predictions on the test set
X_test_scaled = sm.add_constant(X_test_scaled)
X_test_scaled = X_test_scaled[:, selected_features]  # Use the selected features
y_pred = model.predict(X_test_scaled)
y_pred_class = (y_pred > 0.5).astype(int)

# Evaluate the model
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_class))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred_class))

# Final check for multicollinearity using VIF
final_vif_data = calculate_vif(pd.DataFrame(X_train_scaled[:, 1:], columns=iris.feature_names))
print("Final VIF:")
print(final_vif_data)

# Use Lasso regression to handle multicollinearity
lasso_model = LassoCV(cv=5)
lasso_model.fit(X_train_scaled[:, 1:], y_train)  # Exclude the constant term

selected_features_lasso = [True] + list(lasso_model.coef_ != 0)
X_train_scaled_lasso = sm.add_constant(X_train_scaled[:, selected_features_lasso])[:, 1:]

# Continue with the rest of your code
